# BDM550 - Final Project

- Name: Ran Arino
- Student ID: 153073200
- Email: rarino@myseneca.ca
- Course: Predictive Analytics
- Course ID: BDM550NAA.05359.2237
- Professor: Dr. Elnaz Delpisheh

In [138]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

from joblib import dump

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()


### Part 1

In [2]:
"""
# load the dataset
a1 = pd.read_csv('articles1.csv')
a2 = pd.read_csv('articles2.csv')
a3 = pd.read_csv('articles3.csv')

# Define function to load and clean text data
#  this is similar function to the one that I used in Workshop05
def clean_texts(raw_texts):
    # define result
    result = []

    # set of stopwords
    stop_words = set(stopwords.words('english'))
    # set porter stemmers
    porter = nltk.PorterStemmer()

    # traversing all sentences
    for sent in raw_texts:
        # tokenize
        tokens = word_tokenize(sent)
        # defined cleaned sentence
        clean_sent = ''
        # cleaning each sentence
        for w in tokens:
            # if 'w' is one of punctuations, skip to the next word
            if w in string.punctuation:
                continue
            # if 'w' is one of stop words, skip to the next word
            if w.lower() in stop_words:
                continue
            # add stemmed word to clean_sent
            clean_sent += porter.stem(w.lower()) + ' '

        # add clean_sent to result (make sure that the last item is always blank)
        result += [clean_sent[:-1]]

    return result

# cleaning the text & put everything into one variable
texts = []

# traversing all files
for df_sub in [a1, a2, a3]:
    texts += clean_texts(df_sub['content'])
    
# store the cleaned data as new df
titles = list(a1['title']) + list(a2['title']) + list(a3['title'])
dates = list(a1['date']) + list(a2['date']) + list(a3['date'])

df = pd.DataFrame({'title': titles, 'date': dates, 'content': texts})
df = df.dropna(subset=['content'])
df.head()
"""

,title,date,content
0,House Republicans Fret About Winning Their Hea...,2016-12-31,washington — congression republican new fear c...
1,Rift Between Officers and Residents as Killing...,2017-06-19,bullet shell get count blood dri votiv candl b...
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",2017-01-06,walt disney ’ “ bambi ” open 1942 critic prais...
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",2017-04-10,death may great equal ’ necessarili evenhand f...
4,Kim Jong-un Says North Korea Is Preparing to T...,2017-01-02,seoul south korea — north korea ’ leader kim s...


In [147]:
# short cut if 'df.csv' has already been loaded
df = pd.read_csv('df.csv')
#df = df.dropna(subset=['content'])
len(df)

142535

: 

In [16]:
# vectorizing the texts
vect = CountVectorizer()
vect_texts = vect.fit_transform(df['content'])

print(vect_texts.shape)
print(vect_texts[:5].toarray())

(142535, 260539)
[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [17]:
# train the model (20 topics)
lda = LatentDirichletAllocation(n_components=20)
results = lda.fit_transform(vect_texts)

# get topic-word distribution
topic_word_dist = lda.components_
# get document-word distribution
doc_word_dist = lda.transform(vect_texts)

In [139]:
# save two models
dump(lda, 'lda_model.joblib')
dump(vect, 'vectorizer.joblib')

['vectorizer.joblib']

In [62]:
# storing the topic-word distributions
with open('topic_word_dist.txt', 'w', encoding='utf-8') as f:
    # the first line shows vochaburaries
    f.write(" ".join(list(vect.get_feature_names_out())) + "\n")
    # the rest of lines shows distributions
    for i in range(len(topic_word_dist)):
        f.write(" ".join(list([str(i) for i in topic_word_dist[i]])) + "\n")

In [127]:
# storing the document-word distribution
topic_num = len(doc_word_dist[0])
with open('doc_word_dist.txt', 'w', encoding='utf-8') as f:
    # the first line shows vochaburaries
    f.write(" ".join([f'topic{n}' for n in range(1, topic_num+1)]) + "\n")
    # the rest of lines shows distributions
    for i in range(len(doc_word_dist)):
        f.write(" ".join(list([str(i) for i in doc_word_dist[i]])) + "\n")

In [37]:
# show visualization
pyLDAvis.lda_model.prepare(lda, vect_texts, vect)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.099596 -0.138671       1        1  11.114535
3      0.073387 -0.104720       2        1   7.922846
0     -0.000203  0.064202       3        1   6.288741
13    -0.073247 -0.064359       4        1   6.229520
1      0.082496 -0.080897       5        1   6.220142
16     0.127492 -0.038526       6        1   5.793013
17    -0.111733 -0.144602       7        1   5.428025
12     0.106048  0.054692       8        1   4.875799
7      0.071940  0.030623       9        1   4.690420
9     -0.116462 -0.064337      10        1   4.507372
10     0.011806  0.059498      11        1   4.461052
2     -0.075698  0.079229      12        1   4.454782
5      0.091869 -0.029039      13        1   4.350837
18     0.147624  0.002124      14        1   3.894234
15    -0.061888  0.182108      15        1   3.845743
14    -0.041446  0.124409      16        1   3.824964
11    -0.075106  0.025870      17        1   3.549992
19     0.106800  0.062343      18        1   3.060030
4     -0.149678  0.087384      19        1   2.817095
6     -0.014405 -0.107327      20        1   2.670855, topic_info=              Term           Freq          Total Category  logprob  loglift
233279       trump  455735.000000  455735.000000  Default  30.0000  30.0000
49752      clinton  137337.000000  137337.000000  Default  29.0000  29.0000
190951  republican  116088.000000  116088.000000  Default  28.0000  28.0000
181279      presid  193036.000000  193036.000000  Default  27.0000  27.0000
197852        said  567965.000000  567965.000000  Default  26.0000  26.0000
...            ...            ...            ...      ...      ...      ...
83302       follow    5377.941414   62620.078455  Topic20  -5.6663   1.1680
164863       obama    5592.911537   92835.459456  Topic20  -5.6271   0.8134
134491        live    5246.987873   76865.454919  Topic20  -5.6910   0.9384
14739         also    5696.945236  180612.965765  Topic20  -5.6087   0.1663
109206      includ    4790.063510   97031.625284  Topic20  -5.7821   0.6143

[1925 rows x 6 columns], token_table=        Topic      Freq        Term
term                               
1           1  0.008099         000
1           2  0.010369         000
1           3  0.124242         000
1           4  0.036340         000
1           5  0.005565         000
...       ...       ...         ...
256571      3  0.000424  zuckerberg
256571      5  0.001272  zuckerberg
256571     17  0.996009  zuckerberg
257130     19  0.995900       était
257167     19  0.995906         été

[13106 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 4, 1, 14, 2, 17, 18, 13, 8, 10, 11, 3, 6, 19, 16, 15, 12, 20, 5, 7])